In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 읽기
f=open("C:/Users/wjddu/git/BigData/train/train_ori.csv", "r")
dataframe = pd.read_csv(f, header=0)


In [2]:
quantile_25 = np.quantile(dataframe['pow_gen'], 0.25)
quantile_75 = np.quantile(dataframe['pow_gen'],0.75)
IQR = quantile_75 - quantile_25
minimum = quantile_25 - 1.5 * IQR
maximum = quantile_75 + 1.5 * IQR
print(minimum, maximum)
dataframe.drop(dataframe[dataframe.pow_gen >= 1029.33625]. index, inplace = True)

-617.60175 1029.33625


In [3]:
quantile_25 = np.quantile(dataframe['temperature'], 0.25)
quantile_75 = np.quantile(dataframe['temperature'],0.75)
IQR = quantile_75 - quantile_25
minimum = quantile_25 - 1.5 * IQR
maximum = quantile_75 + 1.5 * IQR
print(minimum, maximum)
dataframe.drop(dataframe[dataframe.temperature >= 45.75]. index, inplace = True)

-19.450000000000003 45.75


In [4]:
quantile_25 = np.quantile(dataframe['humidity'], 0.25)
quantile_75 = np.quantile(dataframe['humidity'],0.75)
IQR = quantile_75 - quantile_25
minimum = quantile_25 - 1.5 * IQR
maximum = quantile_75 + 1.5 * IQR
print(minimum, maximum)
dataframe.drop(dataframe[dataframe.humidity >= 148.5]. index, inplace = True)

-7.5 148.5


In [5]:
quantile_25 = np.quantile(dataframe['dew_point'], 0.25)
quantile_75 = np.quantile(dataframe['dew_point'],0.75)
IQR = quantile_75 - quantile_25
minimum = quantile_25 - 1.5 * IQR
maximum = quantile_75 + 1.5 * IQR
print(minimum, maximum)
dataframe.drop(dataframe[dataframe.dew_point >= 46.050000000000004]. index, inplace = True)

-31.150000000000002 46.050000000000004


In [6]:
quantile_25 = np.quantile(dataframe['sol_rad'], 0.25)
quantile_75 = np.quantile(dataframe['sol_rad'],0.75)
IQR = quantile_75 - quantile_25
minimum = quantile_25 - 1.5 * IQR
maximum = quantile_75 + 1.5 * IQR
print(minimum, maximum)
dataframe.drop(dataframe[dataframe.dew_point >= 444.44499999999994]. index, inplace = True)

-266.667 444.44499999999994


In [7]:
quantile_25 = np.quantile(dataframe['cloud'], 0.25)
quantile_75 = np.quantile(dataframe['cloud'],0.75)
IQR = quantile_75 - quantile_25
minimum = quantile_25 - 1.5 * IQR
maximum = quantile_75 + 1.5 * IQR
print(minimum, maximum)
dataframe.drop(dataframe[dataframe.dew_point >= 17.5]. index, inplace = True)

-10.5 17.5


In [8]:
seq= dataframe[['temperature', 'humidity', 'dew_point', 'sol_rad', 'cloud']].to_numpy()

In [9]:
#윈도우 단위로 자르는 함수
def seq2dataset(seq,window, horizon):
    X=[]; Y=[]
    for i in range(len(seq)-(window+horizon)+1):
        x= seq[i:i+window]
        y= seq[i+window+horizon-1]
        X.append(x); Y.append(y)
    return np.array(X), np.array(Y)

# 한 달 정보를 측정하여 다음 날의 정보를 예측
w=30
h=1

X,Y= seq2dataset(seq,w,h)
print(X.shape, Y.shape)

(25523, 30, 5) (25523, 5)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

#훈련 집합과 테스트 집합으로 분할
split = int(len(X)*0.7)
x_train= X[0:split]; y_train = Y[0:split]
x_test= X[split:]; y_test = Y[split:]

In [ ]:
#LSTM 모델 설계와 학습
model=Sequential()
model.add(LSTM(units=128, activation='relu', input_shape=x_train[0].shape))
model.add(Dense(5))
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
hist=model.fit(x_train,y_train, epochs=30, batch_size=16, validation_data=(x_test,y_test), verbose=1)

In [ ]:
#LSTM 모델 평가
ev=model.evaluate(x_test,y_test, verbose=0)
print("손실 함수", ev[0], "MSE:",ev[1])
print(ev[1] ** 0.5)

In [ ]:
from sklearn.metrics import mean_squared_error

RMSE = mean_squared_error(y_test, pred)**0.5
print(RMSE)

In [ ]:
# R2 구하기
from sklearn.metrics import r2_score
r2_y_predict = r2_score(y_test, pred)
print("R2 : ", r2_y_predict)


In [ ]:
#LSTM 모델로 예측 수행
pred=model.predict(x_test)

def MAPE(y, pred) : return np.mean(np.abs((y-pred) / y ) * 100)

print("평균 절댓값 백분율 오차(MAPE)", MAPE(y_test, pred))

In [ ]:
#학습 곡선
plt.rcParams["figure.figsize"] = (14,4)
plt.plot(hist.history['mae'])
plt.plot(hist.history['val_mae'])
plt.title('Model mae')
plt.ylabel('mae')
plt.xlabel('Epoch')
# plt.ylim([120, 800])
plt.legend(['Train', 'Validation'],loc='best')
plt.grid()
plt.show()

In [ ]:
#예측 결과 시각화
plt.rcParams["figure.figsize"] = (20,8)
x_range = range(len(y_test))
plt.plot(x_range, y_test[x_range], color='red')
plt.plot(x_range, pred[x_range], color='blue')
plt.legend(['True prices', 'Predicted prices'], loc='best')
plt.grid()
plt.show()

In [ ]:
#일부 구간을 확대하여 시각화
x_range = range(30,100)
plt.rcParams["figure.figsize"] = (20,8)
plt.plot(x_range, y_test[x_range], color='red')
plt.plot(x_range, pred[x_range], color='blue')
plt.legend(['True prices', 'Predicted prices'], loc='best')
plt.grid()
plt.show()